In [1]:
import numpy as np
import tensorflow as tf

In [2]:
t_min, t_max = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3+2*np.sin(t*5)

def next_batch(batch_size , n_steps):
    t0 = np.random.rand(batch_size,1) * (t_max - t_min - n_steps *resolution)
    Ts = t0 + np.arange(0., n_steps+1) *resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps,1), ys[:, 1:].reshape(-1, n_steps,1)

t = np.linspace(t_min, t_max, int((t_max - t_min / resolution)))
n_steps = 20
t_instance = np.linspace(12.2, 12.2 + resolution * (n_steps +1), n_steps +1)

In [7]:
print(t.shape)
t

(30,)


array([ 0.        ,  1.03448276,  2.06896552,  3.10344828,  4.13793103,
        5.17241379,  6.20689655,  7.24137931,  8.27586207,  9.31034483,
       10.34482759, 11.37931034, 12.4137931 , 13.44827586, 14.48275862,
       15.51724138, 16.55172414, 17.5862069 , 18.62068966, 19.65517241,
       20.68965517, 21.72413793, 22.75862069, 23.79310345, 24.82758621,
       25.86206897, 26.89655172, 27.93103448, 28.96551724, 30.        ])

In [8]:
t_instance

array([12.2  , 12.305, 12.41 , 12.515, 12.62 , 12.725, 12.83 , 12.935,
       13.04 , 13.145, 13.25 , 13.355, 13.46 , 13.565, 13.67 , 13.775,
       13.88 , 13.985, 14.09 , 14.195, 14.3  ])

In [10]:
print(time_series(t).shape)
time_series(t)

(30,)


array([  0.        ,  -1.4956698 ,  -0.98542969,   0.41858562,
         0.77000013,  -0.21216204,  -0.90220748,  -0.01897261,
         1.49086694,   1.43727085,  -0.75611993,  -2.83542147,
        -2.01093157,   1.55149868,   4.22912428,   2.61154232,
        -2.360916  ,  -5.65429883,  -3.22904775,   3.17576695,
         7.09278874,   3.85531419,  -3.98529156,  -8.52646825,
        -4.48438203,   4.77686085,   9.93774955,   5.11263177,
        -5.53633624, -11.3100691 ])

In [3]:
n_steps = 20
n_neurons = 120
n_inputs = 1
n_outputs = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

cell = tf.nn.rnn_cell.BasicRNNCell(num_units = n_neurons, activation = tf.nn.relu)
rnn_outputs, states = tf.nn.dynamic_rnn(cell, X,dtype = tf.float32)

#하나출력을 위한 프로세스
stacked_rnn_outputs = tf.reshape(tensor= rnn_outputs, shape = [-1, n_neurons])
stacked_outputs = tf.layers.dense(stacked_rnn_outputs,units = n_outputs)
predictions = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])

#train
learning_rate = 0.001
n_iterations = 1000
batch_size = 50

mse = tf.losses.mean_squared_error(labels = y, predictions = predictions)
train_op = tf.train.AdamOptimizer(learning_rate).minimize(mse)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for iteration in range(n_iterations):
        batch_x, batch_y = next_batch(batch_size, n_steps)
        sess.run(train_op, feed_dict={X: batch_x, y:batch_y})
        
        if iteration % 100 == 0:
            loss = mse.eval(feed_dict={X:batch_x, y: batch_y})
            print('iteration : {:03d}, loss : {:.4f}'.format(iteration, loss))

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
iteration : 000, loss : 12.9827
iteration : 100, loss : 0.5162
iteration : 200, loss : 0.1709
iteration : 300, loss : 0.0910
iteration : 400, loss : 0.0541
iteration : 500, loss : 0.0483
iteration : 600, loss : 0.0563
iteration : 700, loss : 0.0392
iteration : 800, loss : 0.0531
iteration : 900, loss : 0.0446
